<a href="https://colab.research.google.com/github/RafaelAnga/MachineLearning_Bootcamp/blob/main/natural_language_processing_pipeline_Lemmatization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing

(explain situation ->
explain process -> )
explain pre-processing tools inlcluding pipeline
explain which classification models



## Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from textblob import TextBlob
from nltk.stem import WordNetLemmatizer

from imblearn.pipeline import Pipeline as ImbPipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

## Importing the dataset

In [ ]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os #Libary necesary for accessing the folder
os.chdir('/content/drive/MyDrive/Machine Learning/NLP/DataSets')

os.listdir()

['Restaurant_Reviews.tsv']

In [ ]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3) #quoting = 3 ignores the quotes

## Cleaning the texts

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #Transform words into their base from (loved -> love) -> into Present tense
corpus = []

nltk.download('wordnet')
#iterates indexes
for i in range(0, 1000):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split() #splits review into different words -------> ask chatgpt
  lm = WordNetLemmatizer()
  review = [lm.lemmatize(word) for word in review]
  #Used to remove 'not' from stopwords
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  #review = [ps.stem(word) for word in review if len(word) > 2 and word not in set(all_stopwords)]
  #review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = [re.sub(r'(.)\1+', r'\1', word) for word in review]
  review = [str(TextBlob(word).correct()) for word in review]

  custom_stopwords = {'food', 'place', 'service', 'restaurant', 'menu', 'server', 'staff'}
  review = [word for word in review if word not in custom_stopwords]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
corpus

['now loved this',
 'crust is not god',
 'not taste and the texture wa just nasty',
 'stopped by during the late may bank holiday of rich steve recommendation and loved it',
 'the selection on the wa great and so were the price',
 'now i am getting angry and i want my damn who',
 'honestly it didn t taste that fresh',
 'the potato were like rubber and you could tell they had ben made up ahead of time being kept under a warmer',
 'the cry were great to',
 'a great touch',
 'wa very prompt',
 'would not go back',
 'the cashier had no care what so ever on what i had to say it still ended up being way overpriced',
 'i tried the cape cod revolt chicken with chancery m',
 'i wa disgusted because i wa pretty sure that wa human hair',
 'i wa shocked because no sign indicate cash only',
 'highly recommended',
 'waited wa a little slow in',
 'this is not worth your time let alone vera',
 'did not like at al',
 'the bursitis bah',
 'the for amazing',
 'is also cut',
 'i could care les the interio

## Creating the Bag of Words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 1300)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:,-1].values

In [ ]:
len(X[0])

1300

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Training the pipeline models on the Training set

In [ ]:
# Modified Sampling 900-1100 Pipeline
XGBoost_pipeline = ImbPipeline([
    ('classifier', XGBClassifier())])

#Modified model to randomforest
random_forest_pipeline = ImbPipeline([
    ('classifier', RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42))])

logistic_reg_pipeline = ImbPipeline([
    ('classifier', LogisticRegression())])

naive_bayes_pipeline = ImbPipeline([
    ('classifier', GaussianNB())])

svm_pipeline = ImbPipeline([
    ('classifier', SVC(probability=True))])

gradient_boost_pipeline = ImbPipeline([
    ('classifier', GradientBoostingClassifier(random_state=42))])

knn_pipeline = ImbPipeline([
    ('classifier', KNeighborsClassifier(n_neighbors=5))])

In [ ]:
# Define the pipelines
pipelines_to_compare = {
    'XGBoost Pipeline': XGBoost_pipeline,
    'Random Forest Pipeline': random_forest_pipeline,
    'Logistic Regression Pipeline': logistic_reg_pipeline,
    'Naive Bayes Pipeline': naive_bayes_pipeline,
    'Support Vector Machine Pipeline': svm_pipeline,
    'Gradient Boost Pipeline': gradient_boost_pipeline,
    'KNN Pipeline': knn_pipeline
}

results = {}

for name, pipeline in pipelines_to_compare.items():
    print(f"\nEvaluating {name} on X_train...")

    # Train the pipeline
    pipeline.fit(X_train, y_train)

    # Predict on test data
    y_pred = pipeline.predict(X_train)
    y_pred_proba = pipeline.predict_proba(X_train)[:, 1]

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_train, y_pred)


    # Store results
    results[name] = {
        'Accuracy': accuracy,
    }


# Display results for comparison
results_df = pd.DataFrame(results).T

print("Pipeline Comparison Results:")
display(results_df)


Evaluating XGBoost Pipeline on X_train...

Evaluating Random Forest Pipeline on X_train...

Evaluating Logistic Regression Pipeline on X_train...

Evaluating Naive Bayes Pipeline on X_train...

Evaluating Support Vector Machine Pipeline on X_train...

Evaluating Gradient Boost Pipeline on X_train...

Evaluating KNN Pipeline on X_train...
Pipeline Comparison Results:


,Accuracy
XGBoost Pipeline,0.94625
Random Forest Pipeline,0.99250
Logistic Regression Pipeline,0.97125
Naive Bayes Pipeline,0.90625
Support Vector Machine Pipeline,0.97750
Gradient Boost Pipeline,0.87875
KNN Pipeline,0.78750


## Predicting the Test set results & making the Confusion Matrix

In [ ]:
# Initialize the test results dictionary
test_results = {}

# Evaluate the pipelines on the test set
for name, pipeline in pipelines_to_compare.items():
    print(f"\nEvaluating {name} on X_test...")

    # Predict on the test set
    y_test_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

    # Calculate evaluation metrics
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_auc = roc_auc_score(y_test, y_pred_proba)
    class_report = classification_report(y_test, y_test_pred, output_dict=True)  # Use output_dict for easier DataFrame conversion
    conf_matrix = confusion_matrix(y_test, y_test_pred)

    # Store results
    test_results[name] = {
        'Test Accuracy': test_accuracy,
        'Test AUC': test_auc,
        'Confusion Matrix': conf_matrix,
        'Classification Report': class_report,
    }

    print(f"Accuracy: {test_accuracy}")
    print(f"AUC-ROC: {test_auc}")
    print(f"Classification Report:\n{classification_report(y_test, y_test_pred)}")
    print(f"Confusion Matrix:\n{conf_matrix}")

# Display test results in a DataFrame (summary)
test_results_summary = pd.DataFrame(
    {name: {'Test Accuracy': metrics['Test Accuracy'], 'Test AUC': metrics['Test AUC']}
     for name, metrics in test_results.items()}
).T

print("\nTest Results Summary:")
print(test_results_summary)

# Optional: Save detailed classification reports and confusion matrices
for name, metrics in test_results.items():
    class_report_df = pd.DataFrame(metrics['Classification Report']).transpose()
    print(f"\nDetailed Classification Report for {name}:")
    print(class_report_df)
    print(f"Confusion Matrix for {name}:\n{metrics['Confusion Matrix']}")



Evaluating XGBoost Pipeline on X_test...
Accuracy: 0.77
AUC-ROC: 0.8427584826343709
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.88      0.79        97
           1       0.85      0.67      0.75       103

    accuracy                           0.77       200
   macro avg       0.78      0.77      0.77       200
weighted avg       0.79      0.77      0.77       200

Confusion Matrix:
[[85 12]
 [34 69]]

Evaluating Random Forest Pipeline on X_test...
Accuracy: 0.78
AUC-ROC: 0.8522169952957661
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.88      0.79        97
           1       0.86      0.69      0.76       103

    accuracy                           0.78       200
   macro avg       0.79      0.78      0.78       200
weighted avg       0.79      0.78      0.78       200

Confusion Matrix:
[[85 12]
 [32 71]]

Evaluating Logistic Regression Pipeline on X_test